In [7]:
from direct.showbase.ShowBase import ShowBase
from panda3d.core import WindowProperties,Filename, Texture, PNMImage, GeomVertexFormat, GeomVertexData, Geom, GeomTriangles, GeomNode, GeomVertexWriter,AmbientLight,DirectionalLight
import cv2
import threading
import mediapipe as mp
import math
import numpy as np
import sys
#import time

class MyApp(ShowBase):
    def __init__(self):
        ShowBase.__init__(self)
        
        self.max_dis5_17=0
        self.count=0
        self.x_mid=0
        self.y_mid=0
        self.angle=0
        self.xRotation=0
        self.ring_widthRatio=0
        
        # Set up camera capture
        self.capture = cv2.VideoCapture("test2.mp4")
        parent_node = self.render2d.attachNewNode("ParentNode") 

        # Create a texture to store the captured frame
        self.texture = Texture("CameraTexture")
        self.texture.setWrapU(Texture.WM_clamp)
        self.texture.setWrapV(Texture.WM_clamp)
        self.texture.setMinfilter(Texture.FT_linear)
        self.texture.setMagfilter(Texture.FT_linear)

        # Create a quad to display the captured frame
        format = GeomVertexFormat.get_v3n3t2()
        vdata = GeomVertexData("CameraDisplay", format, Geom.UH_static)
        vdata.setNumRows(4)

        vertex = GeomVertexWriter(vdata, "vertex")
        texcoord = GeomVertexWriter(vdata, "texcoord")

        
        
        vertex.addData3(1, 0, 1)
        vertex.addData3(-1, 0, 1)
        vertex.addData3(-1, 0, -1)
        vertex.addData3(1, 0, -1)

        texcoord.addData2(0, 1)
        texcoord.addData2(1, 1)
        texcoord.addData2(1, 0)
        texcoord.addData2(0, 0)
       
        tris = GeomTriangles(Geom.UH_static)
        tris.addVertices(0, 1, 2)
        tris.addVertices(0, 2, 3)
        tris.closePrimitive()

        geom = Geom(vdata)
        geom.addPrimitive(tris)

        node = GeomNode("CameraDisplay")
        node.addGeom(geom)

        self.camera_display = self.render2d.attachNewNode(node)
        self.camera_display.setTexture(self.texture)
        

        self.camera_display.setBin("fixed", 0)
        self.camera_display.setDepthWrite(True)
        self.camera_display.setDepthTest(True)


        self.camera_display.reparentTo(parent_node)

        
        # Load the .obj file
        self.model = self.loader.loadModel('rings/gold/decorated_ring.glb')  # rings/diamond_ring_-_day_1_3dinktober2019-ring/diamond_ring_-_day_1_3dinktober2019-ring.glb    ----  
        # rings/doji-diamond-ring_org/doji_diamond_ring.glb   ---rings/ring_of_noobs/untitled.glb
        # gamed -- rings/gold/decorated_ring.glb   ---rings/fdy/untitled.glb     --rings/untitled.glb
        bounding_volume = self.model.getBounds()
        min_point = bounding_volume.getMin()
        max_point = bounding_volume.getMax()
        
        self.height = (0.2769268751144409/(max_point[2] - min_point[2]))/1.4
        self.z = (0.2769268751144409/(max_point[1] - min_point[1]))/1.4
        self.width =(0.2769268751144409/(max_point[0] - min_point[0]))/1.4
        




      


        
        # Attach the model to the scene graph
        self.cam.setPos(0,-20,0)
        self.cam.lookAt(0,0,0)
        
        self.model.setBin("fixed", 1)
        self.model.setDepthWrite(True)
        self.model.setDepthTest(True)
        texture = self.loader.loadTexture('rings/gold/textures/DefaultMaterial_AO.jpeg')  #-rings\doji_diamond_ring\textures\Material_2_baseColor.png
        #rings/ring_of_noobs/textures/None_normal.jpeg
        #texture gamed --rings/gold/textures/DefaultMaterial_AO.jpeg      rings/fdy/textures/Material_normal.png
        self.model.setTexture(texture)
        ambient_light = AmbientLight("ambient_light")
        ambient_light.setColor((1,1, 1, 1))
       
        ambient_node = self.render.attachNewNode(ambient_light)
        self.model.setLight(ambient_node)

        directional_light = DirectionalLight("directional_light")
        directional_light.setColor((1, 1, 1, 1))
        directional_light.setDirection((-1, -1, -1))  # Light direction pointing downward
        directional_node = self.render.attachNewNode(directional_light)
        self.model.setLight(directional_node)





        self.model.reparentTo(self.camera_display)

        self.shshs=True
        self.accept("escape", self.closeApp)

        self.texture_lock = threading.Lock()

        # Start the update loop in a separate thread
        self.update_thread = threading.Thread(target=self.update_loop)
        self.update_thread.start()
        
        
    def getEvery(self,image):
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mp_hands = mp.solutions.hands
        mp_drawing = mp.solutions.drawing_utils


        with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5) as hands:
                # Process the image with MediaPipe
                results = hands.process(image)
                
                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_landmarks:
                        
                        if hand_landmarks.landmark[20].x > hand_landmarks.landmark[8].x:
                            hand_type = "Back Hand"
                        else :
                            hand_type = "Front Hand"


                        # Add the ring image on the middle finger (landmark index 12)
                        middle_finger_landmark13 = hand_landmarks.landmark[13]
                        middle_finger___landmark14 = hand_landmarks.landmark[14]
                        middle_finger___landmark17 = hand_landmarks.landmark[17]
                        middle_finger___landmark9= hand_landmarks.landmark[9]
                        middle_finger___landmark5= hand_landmarks.landmark[5]
                        middle_finger___landmark15= hand_landmarks.landmark[15]
                        x13 = middle_finger_landmark13.x #* image.shape[1])
                        y13 = middle_finger_landmark13.y #* image.shape[0])
                        x14 = middle_finger___landmark14.x #* image.shape[1])
                        y14 = middle_finger___landmark14.y #* image.shape[0])



                        # get position
                        # print("x15=",x15," y=15",y15)
                        # print("x14=",x14," y=14",y14)
                        try:
                            #check x if exist in image
                            self.x_mid=((x14+x13)/2)            
                        except ZeroDivisionError:
                            print("not all hand in image")
                            return False
                        # rotation part 
                        rotate = self.calculate_rotation_angle(hand_landmarks.landmark[13],hand_landmarks.landmark[14])
                        ##################
                        #print(self.ring_width)

                        if hand_type == "Back Hand":
                            self.xRotation=0
                           
                            
                           
                            # y position of ring
                            
                            self.y_mid=((y14+y13 ) /2)-.03
                            #rotate ring in y axis
                            self.angle = int(90-rotate)
                            self.ring_width =  abs(y13-y14) /1.15
                            if self.ring_width<.09:
                                
                                self.ring_width =  abs(x13-x14) /2.5
                            
                            if self.angle>30:
                                self.x_mid-=.05
                            elif self.angle<-30:
                                self.x_mid+=.02
                            elif self.angle>10:
                                self.x_mid-=.02
                            self.ringHeight=1.5
                            self.zOfScale=5
                        else:
                            
                            self.ring_width =  abs(y13-y14)/1
                            if self.ring_width<.06:
                                
                                self.ring_width =  abs(x13-x14) /3

                            #self.x_mid-=.01 
                            self.xRotation=180
                            # y position of ring
                            
                            
                            self.y_mid=((y14+y13 ) /2)-.01
                            #rotate ring in y axis
                            self.angle = 180-int(80-rotate)
                            
                            if self.angle>190:
                                self.x_mid+=.01
                                
                                self.angle-= 30
                            
                            self.ringHeight=1
                            self.zOfScale=-5
             
                        #convert from 0 -> 1  to -1 -> 1
                        self.x_mid = (self.x_mid * 2) - 1
                        self.y_mid = (self.y_mid * 2) - 1
                        
                        
                    
                            
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)                
        cv2.imwrite("yes.jpg",image)       
          
        return True

    def update_loop(self):
        self.ringx=0
        self.ringy=0
        while True:
            # Capture frame from the camera
            ret, frame = self.capture.read()

            
            if ret:
                frame = cv2.flip(frame, 1)
                height, width, channels = frame.shape
                self.set_window_size(width=width, height=height)
                # Convert the frame to RGB format
                
                
                check=self.getEvery(frame)
                
                
                if check:
                    if self.ringx==0:
                        self.ringx= self.x_mid
                        self.ringy= self.y_mid
                    self.ringx= self.x_mid *.7 + self.ringx*.3
                    self.ringy= self.y_mid *.7 + self.ringy*.3
                    self.anglee=self.angle
                    self.xRotationn=self.xRotation
                    
                    self.ring_widthRatiooo=self.ring_width*12
            

                

                image = PNMImage(Filename("yes.jpg"))
                
                
                self.model.setPos(-1*self.ringx,0, -1*self.ringy)
                
                self.model.setScale(self.ring_widthRatiooo*self.height,4*self.z,self.ringHeight*self.width)
                
                

                self.model.setHpr(self.xRotationn ,self.zOfScale,self.anglee)  #self.xRotationn  
                
                # Acquire the lock before updating the texture
                self.texture_lock.acquire()

                # Set the PNMImage as the texture's RAM image
                self.texture.load(image)

                # Release the lock after updating the texture
                self.texture_lock.release()
                self.count+=1
            
            else:
                self.closeApp()
                sys.exit()

    def set_window_size(self, width, height):
        props = WindowProperties()
        props.setSize(width, height)
        self.win.requestProperties(props)                
    def calculate_rotation_angle(self,landmark, reference_point):
        dx = landmark.x - reference_point.x
        dy = landmark.y - reference_point.y
        angle_rad = math.atan2(dy, dx)
        angle_degrees = math.degrees(angle_rad)
        return round(angle_degrees, 2)              

    def closeApp(self):
        self.capture.release()
        self.destroy()
        sys.exit()
app = MyApp()
app.run()


SystemExit: 